In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Building a RAG Pipeline from Scratch: From Documents to Answers

*Part 3 of the Vizuara series on Context Engineering for LLMs*

**Estimated time: 40 minutes**

In the previous notebooks, we explored what context engineering is and how the context window works. Now we tackle the single most common context engineering pattern in production: **Retrieval-Augmented Generation (RAG)**.

By the end of this notebook, you will have built a complete RAG pipeline — from raw text to assembled context — entirely from scratch. No black-box vector databases. No magic APIs. Just numpy, a small embedding model, and your own code.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/context-engineering-for-llms/practice/3/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Code Walkthrough: Install Imports
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_install_imports.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 0. Setup and Installations

Let us install the libraries we need. We use `sentence-transformers` for embedding text (this is a **tool**, not the concept we are learning — the concept is the retrieval pipeline itself).

In [ ]:
!pip install -q sentence-transformers matplotlib numpy scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import textwrap
import re
import time
from typing import List, Tuple, Dict, Optional

# Reproducibility
np.random.seed(42)

# Plotting defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

print("Setup complete!")

In [ ]:
#@title 🎧 Listen: Why Rag
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_why_rag.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

Imagine you are a brilliant student walking into an open-book exam. Your intelligence is fixed — you cannot suddenly become smarter. But your performance depends on **what notes you brought** and **how fast you can find the right page**.

Large Language Models face the same challenge. Their "intelligence" (weights) is frozen after training. But we can dramatically improve their answers by retrieving the right information and injecting it into their context window. This is RAG.

**Here is the problem RAG solves:** An LLM trained in 2024 does not know about events in 2025. It does not know your company's internal documentation. It does not know the specifics of your codebase. But if we *retrieve* the right documents and *augment* the prompt with them, the LLM can answer as if it knew everything.

The RAG pipeline has seven steps:

1. User sends a query
2. Query is converted to an embedding (a vector of numbers)
3. Compare against pre-computed document chunk embeddings
4. Retrieve top-K most similar chunks
5. Pass through a reranker for a second relevance check
6. Inject selected chunks into the context window
7. LLM generates a response using training knowledge + retrieved context

We will build steps 1 through 6 from scratch. Step 7 just needs an LLM API call — the interesting engineering is everything before it.

In [ ]:
#@title 🎧 Listen: Rag Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_rag_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

### The Open-Book Exam Analogy

Let us make the three key design decisions in RAG concrete with our exam analogy:

| RAG Concept | Exam Analogy | What Happens If You Get It Wrong |
|---|---|---|
| **Chunk size** | Size of your note cards | Too small = confetti (no context). Too large = wastes space. |
| **Top-K** | How many cards you bring | Too few = miss the answer. Too many = noise drowns signal. |
| **Reranking** | Sorting best card to top | Without it, the answer is buried in the middle of your notes. |

The sweet spot for chunk size is typically **400-1000 tokens** per chunk. We will experiment with this later and see the difference ourselves.

In [ ]:
#@title 🎧 Listen: Think About This
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_think_about_this.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🤔 Think About This

Before we write any code, consider:

- Why can we not just dump the entire knowledge base into the context window? (Hint: token limits, cost, and the "Lost in the Middle" problem)
- Why do we need embeddings at all? Could we just use keyword search? (Hint: "automobile" and "car" share zero keywords but mean the same thing)
- Why would we need a reranker if we already have similarity scores? (Hint: embedding similarity is a rough first pass, not a precise relevance judgment)

Take a moment to think about these. Then let us build.

In [ ]:
#@title 🎧 Listen: Cosine Sim Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_cosine_sim_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Cosine Similarity — The Heart of Retrieval

The core mathematical operation in RAG is **cosine similarity**. Given a query vector $\mathbf{q}$ and a document vector $\mathbf{d}$, cosine similarity measures how "aligned" they are:

$$\text{sim}(\mathbf{q}, \mathbf{d}) = \frac{\mathbf{q} \cdot \mathbf{d}}{||\mathbf{q}|| \; ||\mathbf{d}||}$$

**What does this mean computationally?**

1. Compute the dot product of the two vectors (multiply element-wise, then sum)
2. Compute the magnitude (length) of each vector
3. Divide the dot product by the product of the magnitudes

The result is always between -1 and 1:
- **1.0** = vectors point in the same direction (highly similar)
- **0.0** = vectors are perpendicular (unrelated)
- **-1.0** = vectors point in opposite directions (opposite meaning)

In [ ]:
#@title 🎧 Listen: Cosine Sim Ex1
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_cosine_sim_ex1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Worked Example 1: Perfectly Aligned Vectors

Let us compute this by hand first, then verify with code.

$\mathbf{q} = [1, 2, 3]$, $\mathbf{d} = [2, 4, 6]$

**Step 1 — Dot product:** $1 \times 2 + 2 \times 4 + 3 \times 6 = 2 + 8 + 18 = 28$

**Step 2 — Magnitudes:** $||\mathbf{q}|| = \sqrt{1^2 + 2^2 + 3^2} = \sqrt{14} \approx 3.74$, $||\mathbf{d}|| = \sqrt{2^2 + 4^2 + 6^2} = \sqrt{56} \approx 7.48$

**Step 3 — Divide:** $\text{sim} = \frac{28}{3.74 \times 7.48} \approx 1.0$

A similarity of **1.0** — perfectly aligned! Notice that $\mathbf{d} = 2 \times \mathbf{q}$. Cosine similarity does not care about magnitude, only direction. These vectors point the same way.

In [ ]:
# Worked Example 1: Perfectly aligned vectors
q = np.array([1, 2, 3], dtype=float)
d = np.array([2, 4, 6], dtype=float)

dot_product = np.dot(q, d)
magnitude_q = np.linalg.norm(q)
magnitude_d = np.linalg.norm(d)
similarity = dot_product / (magnitude_q * magnitude_d)

print(f"q = {q}")
print(f"d = {d}")
print(f"Dot product: {dot_product:.1f}")
print(f"||q|| = {magnitude_q:.4f}")
print(f"||d|| = {magnitude_d:.4f}")
print(f"Cosine similarity = {dot_product:.1f} / ({magnitude_q:.4f} × {magnitude_d:.4f}) = {similarity:.4f}")
print(f"\nVerdict: {'Perfectly aligned!' if similarity > 0.99 else 'Not aligned'}")

In [ ]:
#@title 🎧 Listen: Cosine Sim Ex2
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_cosine_sim_ex2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Worked Example 2: Barely Related Vectors

Now try $\mathbf{q} = [1, 2, 3]$, $\mathbf{d'} = [3, -1, 0]$

**Dot product:** $1 \times 3 + 2 \times (-1) + 3 \times 0 = 3 - 2 + 0 = 1$

**Magnitudes:** $||\mathbf{q}|| = \sqrt{14} \approx 3.74$, $||\mathbf{d'}|| = \sqrt{9 + 1 + 0} = \sqrt{10} \approx 3.16$

**Similarity:** $\frac{1}{3.74 \times 3.16} \approx 0.08$

A similarity of **0.08** — barely related at all. We would not retrieve this document.

In [ ]:
# Worked Example 2: Barely related vectors
d_prime = np.array([3, -1, 0], dtype=float)

dot_product_2 = np.dot(q, d_prime)
magnitude_d_prime = np.linalg.norm(d_prime)
similarity_2 = dot_product_2 / (magnitude_q * magnitude_d_prime)

print(f"q  = {q}")
print(f"d' = {d_prime}")
print(f"Dot product: {dot_product_2:.1f}")
print(f"||d'|| = {magnitude_d_prime:.4f}")
print(f"Cosine similarity = {dot_product_2:.1f} / ({magnitude_q:.4f} × {magnitude_d_prime:.4f}) = {similarity_2:.4f}")
print(f"\nVerdict: {'Barely related' if similarity_2 < 0.2 else 'Related'}")

In [ ]:
#@title 🎧 What to Look For: Cosine Sim Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_cosine_sim_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: What Cosine Similarity Looks Like

Let us visualize both examples as 2D vectors (projecting to 2D for visibility) to build geometric intuition.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Helper to draw a vector
def draw_vector(ax, vec, label, color, offset=(0.1, 0.1)):
    ax.annotate('', xy=vec, xytext=(0, 0),
                arrowprops=dict(arrowstyle='->', color=color, lw=2.5))
    ax.annotate(label, xy=vec + np.array(offset), fontsize=13,
                fontweight='bold', color=color)

# Example 1: q=[1,2] vs d=[2,4] (perfectly aligned in 2D)
ax1 = axes[0]
q_2d = np.array([1, 2])
d_2d = np.array([2, 4])
draw_vector(ax1, q_2d, 'q = [1,2]', '#2196F3', offset=(0.1, -0.3))
draw_vector(ax1, d_2d, 'd = [2,4]', '#4CAF50', offset=(0.1, -0.3))
ax1.set_xlim(-0.5, 5)
ax1.set_ylim(-0.5, 5)
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)
ax1.set_title(f'Cosine Similarity = 1.00\n(Perfectly Aligned)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Dimension 1')
ax1.set_ylabel('Dimension 2')

# Example 2: q=[1,2] vs d'=[3,-1] (barely related in 2D)
ax2 = axes[1]
q_2d_b = np.array([1, 2])
d_prime_2d = np.array([3, -1])
draw_vector(ax2, q_2d_b, 'q = [1,2]', '#2196F3', offset=(0.1, 0.15))
draw_vector(ax2, d_prime_2d, "d' = [3,-1]", '#F44336', offset=(0.1, -0.3))

# Draw the angle arc
angle_q = np.degrees(np.arctan2(q_2d_b[1], q_2d_b[0]))
angle_d = np.degrees(np.arctan2(d_prime_2d[1], d_prime_2d[0]))
arc = mpatches.Arc((0, 0), 2.0, 2.0, angle=0,
                    theta1=min(angle_d, angle_q), theta2=max(angle_d, angle_q),
                    color='gray', lw=1.5, linestyle='--')
ax2.add_patch(arc)
ax2.annotate('θ ≈ 85°', xy=(1.2, 0.4), fontsize=11, color='gray')

ax2.set_xlim(-0.5, 4)
ax2.set_ylim(-1.5, 3)
ax2.set_aspect('equal')
ax2.grid(True, alpha=0.3)
ax2.set_title(f'Cosine Similarity ≈ 0.08\n(Barely Related)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Dimension 1')
ax2.set_ylabel('Dimension 2')

plt.tight_layout()
plt.suptitle('Cosine Similarity: Direction Matters, Not Magnitude',
             fontsize=15, fontweight='bold', y=1.03)
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Knowledge Base
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_knowledge_base.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

Now we build the full RAG pipeline piece by piece. Each component is self-contained and testable.

### 4.1 The Knowledge Base

First, let us create our knowledge base — a collection of documents about machine learning concepts. We include both short definitions and longer explanatory paragraphs, because real knowledge bases have documents of varying lengths.

In [ ]:
# --- Short definitions (1 sentence each) ---
short_docs = [
    "Batch normalization normalizes layer inputs to stabilize training and allow higher learning rates.",
    "Dropout randomly deactivates neurons during training to prevent overfitting and improve generalization.",
    "The transformer architecture uses self-attention mechanisms to process sequences in parallel, replacing recurrence.",
    "Learning rate scheduling adjusts the optimization step size during training to improve convergence.",
    "Residual connections help gradients flow through deep networks by adding skip connections between layers.",
    "Attention mechanisms allow models to focus on relevant parts of the input, weighting importance dynamically.",
    "Convolutional neural networks use local receptive fields and weight sharing for spatial data like images.",
    "Recurrent neural networks process sequential data through hidden states that carry information across time steps.",
    "Generative adversarial networks use a generator and discriminator trained in competition to produce realistic data.",
    "Transfer learning reuses knowledge from pre-trained models for new tasks, reducing data and compute requirements.",
]

# --- Longer explanatory paragraphs (2-3 sentences each) ---
long_docs = [
    "Batch normalization was introduced by Ioffe and Szegedy in 2015. It works by normalizing the inputs to each layer so that they have zero mean and unit variance. This reduces internal covariate shift, stabilizes the training process, and allows the use of much higher learning rates without the risk of divergence.",

    "Dropout was proposed by Srivastava et al. in 2014 as a simple yet effective regularization technique. During training, each neuron is randomly set to zero with probability p (typically 0.5). This prevents neurons from co-adapting and forces the network to learn redundant representations, which improves generalization to unseen data.",

    "The transformer architecture, introduced in the landmark paper 'Attention Is All You Need' by Vaswani et al. in 2017, replaced recurrence entirely with self-attention. Each token in a sequence attends to every other token, computing attention weights that determine how much to focus on each position. This parallelism enables dramatically faster training compared to RNNs and LSTMs.",

    "Residual connections, or skip connections, were introduced in ResNet by He et al. in 2015. They allow the gradient to flow directly through the network by adding the input of a block to its output. This simple technique enabled training of networks with over 100 layers, which was previously impossible due to vanishing gradients.",

    "Transfer learning has become the default paradigm in modern deep learning. Instead of training a model from scratch on a small dataset, practitioners start with a model pre-trained on a large dataset like ImageNet or Common Crawl. The pre-trained weights capture general features that transfer well to downstream tasks, dramatically reducing the amount of task-specific data needed.",
]

# Combine into a single knowledge base
knowledge_base = short_docs + long_docs

print(f"Knowledge base: {len(knowledge_base)} documents")
print(f"  - {len(short_docs)} short definitions")
print(f"  - {len(long_docs)} longer paragraphs")
print(f"\nExample short doc:\n  '{short_docs[0]}'")
print(f"\nExample long doc:\n  '{long_docs[0][:120]}...'")

In [ ]:
#@title 🎧 Code Walkthrough: Text Chunking
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_text_chunking.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Text Chunking — Three Strategies

Chunking is how we break documents into retrievable pieces. The chunk size determines the granularity of retrieval — too small and each chunk lacks context, too large and we waste precious context window space.

We implement three strategies:

In [ ]:
def chunk_fixed_size(text: str, chunk_size: int = 200, overlap: int = 50) -> List[str]:
    """
    Strategy 1: Fixed-size chunking.
    Split text into chunks of exactly `chunk_size` characters with overlap.
    Simple but can break mid-sentence.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        if chunk:  # Skip empty chunks
            chunks.append(chunk)
        start = end - overlap  # Slide window with overlap
    return chunks


def chunk_by_sentences(text: str, sentences_per_chunk: int = 3) -> List[str]:
    """
    Strategy 2: Sentence-based chunking.
    Split at sentence boundaries and group N sentences per chunk.
    Respects natural language boundaries.
    """
    # Split on sentence-ending punctuation followed by space or end
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    sentences = [s for s in sentences if s.strip()]

    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = ' '.join(sentences[i:i + sentences_per_chunk]).strip()
        if chunk:
            chunks.append(chunk)
    return chunks


def chunk_by_paragraphs(text: str) -> List[str]:
    """
    Strategy 3: Paragraph/section-based chunking.
    Split at double newlines (paragraph boundaries).
    Preserves semantic coherence within each chunk.
    """
    paragraphs = re.split(r'\n\s*\n', text.strip())
    chunks = [p.strip() for p in paragraphs if p.strip()]
    return chunks


print("Three chunking strategies defined!")
print("  1. chunk_fixed_size(text, chunk_size, overlap)")
print("  2. chunk_by_sentences(text, sentences_per_chunk)")
print("  3. chunk_by_paragraphs(text)")

In [ ]:
#@title 🎧 What to Look For: Chunking Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_chunking_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: How Chunking Strategies Differ

Let us see how each strategy splits the same document. This is where intuition meets reality.

In [ ]:
# A longer sample document to chunk
sample_document = """Batch normalization was introduced by Ioffe and Szegedy in 2015. It works by normalizing the inputs to each layer so that they have zero mean and unit variance. This reduces internal covariate shift, stabilizes the training process, and allows the use of much higher learning rates without the risk of divergence.

The key insight is that the distribution of each layer's inputs changes during training, as the parameters of the previous layers change. This internal covariate shift makes training difficult because each layer must continuously adapt to a new input distribution.

By normalizing inputs, batch normalization allows each layer to learn on a more stable distribution. During inference, the running mean and variance computed during training are used instead of batch statistics, ensuring consistent behavior."""

strategies = {
    'Fixed-size (200 chars)': chunk_fixed_size(sample_document, chunk_size=200, overlap=50),
    'Sentence-based (2 per chunk)': chunk_by_sentences(sample_document, sentences_per_chunk=2),
    'Paragraph-based': chunk_by_paragraphs(sample_document),
}

fig, axes = plt.subplots(len(strategies), 1, figsize=(14, 10))
colors_list = ['#2196F3', '#4CAF50', '#FF9800']

for idx, (name, chunks) in enumerate(strategies.items()):
    ax = axes[idx]
    for i, chunk in enumerate(chunks):
        # Show each chunk as a colored bar
        wrapped = textwrap.fill(chunk[:80] + ('...' if len(chunk) > 80 else ''), width=70)
        ax.barh(i, len(chunk), color=colors_list[idx], alpha=0.7, edgecolor='white', height=0.7)
        ax.text(5, i, f'Chunk {i+1}: "{chunk[:55]}..."' if len(chunk) > 55 else f'Chunk {i+1}: "{chunk}"',
                va='center', fontsize=9, fontweight='bold')

    ax.set_title(f'{name}  →  {len(chunks)} chunks', fontsize=13, fontweight='bold')
    ax.set_xlabel('Characters per chunk')
    ax.set_yticks([])
    ax.invert_yaxis()

plt.tight_layout()
plt.suptitle('Same Document, Three Chunking Strategies',
             fontsize=15, fontweight='bold', y=1.02)
plt.show()

# Print statistics
for name, chunks in strategies.items():
    sizes = [len(c) for c in chunks]
    print(f"\n{name}:")
    print(f"  Chunks: {len(chunks)}, Avg size: {np.mean(sizes):.0f} chars, "
          f"Min: {min(sizes)}, Max: {max(sizes)}")

In [ ]:
#@title 🎧 Code Walkthrough: Embedding
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_embedding.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Embedding with Sentence-Transformers

Now we convert text into vectors. We use `all-MiniLM-L6-v2` — a small, fast model that produces 384-dimensional embeddings. This is a **tool**, not the concept. The concept is what we do with these embeddings.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a small, fast embedding model (runs on CPU, no GPU needed)
print("Loading embedding model (this takes ~10 seconds on first run)...")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"Model loaded! Embedding dimension: {embed_model.get_sentence_embedding_dimension()}")

# Embed our entire knowledge base
print(f"\nEmbedding {len(knowledge_base)} documents...")
start_time = time.time()
doc_embeddings = embed_model.encode(knowledge_base, show_progress_bar=True)
doc_embeddings = np.array(doc_embeddings)
elapsed = time.time() - start_time

print(f"\nDone in {elapsed:.2f}s")
print(f"Embedding matrix shape: {doc_embeddings.shape}")
print(f"  → {doc_embeddings.shape[0]} documents, {doc_embeddings.shape[1]} dimensions each")

Let us verify: each document is now a vector of 384 numbers. Let us peek at one.

In [ ]:
# Look at the first document's embedding
print(f"Document: '{knowledge_base[0][:60]}...'")
print(f"\nEmbedding (first 20 values of 384):")
print(f"  {doc_embeddings[0][:20]}")
print(f"\nEmbedding norm: {np.linalg.norm(doc_embeddings[0]):.4f}")
print(f"  (Sentence-transformers normalizes to unit length by default)")

In [ ]:
#@title 🎧 Before You Start: Todo Cosine Sim
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_todo_cosine_sim.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Cosine Similarity from Scratch

Now for the core algorithm. We implement cosine similarity manually — no sklearn, no scipy. Just numpy.

### 🔧 TODO 1: Implement Cosine Similarity

In [ ]:
def cosine_similarity(query_vec: np.ndarray, doc_vecs: np.ndarray) -> np.ndarray:
    """
    Compute cosine similarity between a query vector and multiple document vectors.

    Args:
        query_vec: Shape (embedding_dim,) — a single query embedding
        doc_vecs:  Shape (num_docs, embedding_dim) — matrix of document embeddings

    Returns:
        Shape (num_docs,) — cosine similarity score for each document

    Formula: sim(q, d) = (q · d) / (||q|| * ||d||)

    Requirements:
        - Use ONLY numpy (no sklearn, no scipy)
        - Must handle the case where query_vec is 1D
        - Must handle batched document vectors (2D matrix)
    """
    # ============ TODO ============
    # Step 1: Compute the dot product between query and all docs
    #         Hint: matrix multiplication — doc_vecs @ query_vec gives (num_docs,)
    # Step 2: Compute the magnitude (L2 norm) of the query vector
    # Step 3: Compute the magnitude of each document vector
    # Step 4: Divide dot products by product of magnitudes
    # Step 5: Handle edge case — avoid division by zero
    # ==============================

    dot_products = ???  # YOUR CODE HERE
    query_norm = ???    # YOUR CODE HERE
    doc_norms = ???     # YOUR CODE HERE
    similarities = ???  # YOUR CODE HERE

    return similarities

In [ ]:
#@title 🎧 Listen: After Todo Cosine Sim
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_after_todo_cosine_sim.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ✅ Verification: Run this cell to check your implementation

# Test 1: Identical vectors should have similarity 1.0
test_q = np.array([1.0, 0.0, 0.0])
test_d = np.array([[1.0, 0.0, 0.0]])
result = cosine_similarity(test_q, test_d)
assert np.allclose(result, [1.0], atol=1e-6), f"Test 1 failed: expected [1.0], got {result}"

# Test 2: Orthogonal vectors should have similarity 0.0
test_d2 = np.array([[0.0, 1.0, 0.0]])
result2 = cosine_similarity(test_q, test_d2)
assert np.allclose(result2, [0.0], atol=1e-6), f"Test 2 failed: expected [0.0], got {result2}"

# Test 3: Opposite vectors should have similarity -1.0
test_d3 = np.array([[-1.0, 0.0, 0.0]])
result3 = cosine_similarity(test_q, test_d3)
assert np.allclose(result3, [-1.0], atol=1e-6), f"Test 3 failed: expected [-1.0], got {result3}"

# Test 4: Batched — should work with multiple documents at once
test_d4 = np.array([[1.0, 0.0, 0.0],
                     [0.0, 1.0, 0.0],
                     [-1.0, 0.0, 0.0]])
result4 = cosine_similarity(test_q, test_d4)
assert result4.shape == (3,), f"Test 4 failed: expected shape (3,), got {result4.shape}"
assert np.allclose(result4, [1.0, 0.0, -1.0], atol=1e-6), f"Test 4 failed: expected [1, 0, -1], got {result4}"

# Test 5: Worked example from the article
q_article = np.array([1.0, 2.0, 3.0])
d_article = np.array([[2.0, 4.0, 6.0],
                       [3.0, -1.0, 0.0]])
result5 = cosine_similarity(q_article, d_article)
assert np.allclose(result5[0], 1.0, atol=1e-4), f"Test 5a failed: expected ~1.0, got {result5[0]}"
assert np.allclose(result5[1], 0.0845, atol=0.01), f"Test 5b failed: expected ~0.08, got {result5[1]}"

print("✅ All 5 tests passed! Your cosine similarity implementation is correct.")
print(f"   Article example: sim(q, d) = {result5[0]:.4f}, sim(q, d') = {result5[1]:.4f}")

💡 **Key insight:** When vectors are already L2-normalized (which sentence-transformers does by default), cosine similarity simplifies to just the dot product:

$$\text{sim}(\mathbf{q}, \mathbf{d}) = \mathbf{q} \cdot \mathbf{d} \quad \text{(when } ||\mathbf{q}|| = ||\mathbf{d}|| = 1\text{)}$$

This is why normalized embeddings are so popular — retrieval becomes a single matrix multiplication.

In [ ]:
#@title 🎧 Code Walkthrough: Vector Store
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_vector_store.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Building a Vector Store

Now let us build a simple vector store — a class that holds document embeddings and supports top-K search.

In [ ]:
class SimpleVectorStore:
    """
    A minimal vector store that stores document embeddings
    and supports similarity search.

    This is what Pinecone, Weaviate, and ChromaDB do at their core,
    but we build it ourselves to understand the mechanics.
    """

    def __init__(self):
        self.documents: List[str] = []
        self.embeddings: Optional[np.ndarray] = None
        self.metadata: List[Dict] = []

    def add_documents(self, documents: List[str], embeddings: np.ndarray,
                      metadata: Optional[List[Dict]] = None):
        """Add documents and their embeddings to the store."""
        self.documents.extend(documents)

        if self.embeddings is None:
            self.embeddings = embeddings
        else:
            self.embeddings = np.vstack([self.embeddings, embeddings])

        if metadata:
            self.metadata.extend(metadata)
        else:
            self.metadata.extend([{}] * len(documents))

        print(f"Added {len(documents)} documents. Total: {len(self.documents)}")

    def search(self, query_embedding: np.ndarray, top_k: int = 5) -> List[Tuple[str, float, int]]:
        """
        Search for the top-K most similar documents.

        Returns: List of (document_text, similarity_score, index) tuples,
                 sorted by similarity (highest first).
        """
        if self.embeddings is None:
            return []

        # Compute cosine similarity using our function
        similarities = cosine_similarity(query_embedding, self.embeddings)

        # Get top-K indices (sorted by similarity, descending)
        top_indices = np.argsort(similarities)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append((
                self.documents[idx],
                float(similarities[idx]),
                int(idx)
            ))

        return results

    def __len__(self):
        return len(self.documents)

    def __repr__(self):
        return f"SimpleVectorStore({len(self.documents)} documents, {self.embeddings.shape[1]}d embeddings)"


# Create our vector store and add the knowledge base
vector_store = SimpleVectorStore()
vector_store.add_documents(
    documents=knowledge_base,
    embeddings=doc_embeddings,
    metadata=[{"type": "short" if i < len(short_docs) else "long",
               "index": i} for i in range(len(knowledge_base))]
)
print(f"\n{vector_store}")

In [ ]:
#@title 🎧 Code Walkthrough: Test Retrieval
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_test_retrieval.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Let Us Test Retrieval!

Time to see if our pipeline actually works. Let us query for different ML concepts and see what comes back.

In [ ]:
# Define some test queries
test_queries = [
    "How does batch normalization work?",
    "What prevents neural networks from overfitting?",
    "How do transformers process sequences?",
    "What are skip connections and why do they matter?",
    "How can I use a pre-trained model for a new task?",
]

print("=" * 80)
print("RETRIEVAL TEST: Top-3 results for each query")
print("=" * 80)

for query in test_queries:
    # Embed the query
    query_embedding = embed_model.encode(query)

    # Search
    results = vector_store.search(query_embedding, top_k=3)

    print(f"\n🔍 Query: \"{query}\"")
    print("-" * 60)
    for rank, (doc, score, idx) in enumerate(results, 1):
        doc_preview = doc[:90] + "..." if len(doc) > 90 else doc
        print(f"  {rank}. [{score:.4f}] {doc_preview}")
    print()

In [ ]:
#@title 🎧 What to Look For: Similarity Heatmap
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_similarity_heatmap.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Similarity Matrix Heatmap

Let us visualize the similarity between all queries and all documents. This gives us a bird's-eye view of how well embedding-based retrieval captures semantic relationships.

In [ ]:
# Embed all test queries
query_embeddings = embed_model.encode(test_queries)

# Compute full similarity matrix: (num_queries × num_docs)
similarity_matrix = np.array([
    cosine_similarity(q_emb, doc_embeddings)
    for q_emb in query_embeddings
])

# Create short labels
query_labels = [q[:35] + '...' if len(q) > 35 else q for q in test_queries]
doc_labels = [d[:30] + '...' if len(d) > 30 else d for d in knowledge_base]

fig, ax = plt.subplots(figsize=(16, 7))

im = ax.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)
plt.colorbar(im, ax=ax, label='Cosine Similarity', shrink=0.8)

# Labels
ax.set_xticks(range(len(doc_labels)))
ax.set_xticklabels(doc_labels, rotation=45, ha='right', fontsize=8)
ax.set_yticks(range(len(query_labels)))
ax.set_yticklabels(query_labels, fontsize=10)

ax.set_xlabel('Documents', fontsize=12)
ax.set_ylabel('Queries', fontsize=12)
ax.set_title('Query–Document Similarity Matrix\n(Brighter = More Similar)',
             fontsize=14, fontweight='bold')

# Annotate cells with similarity values
for i in range(len(test_queries)):
    for j in range(len(knowledge_base)):
        val = similarity_matrix[i, j]
        if val > 0.4:
            ax.text(j, i, f'{val:.2f}', ha='center', va='center',
                    fontsize=7, fontweight='bold', color='white' if val > 0.6 else 'black')

plt.tight_layout()
plt.show()

# Highlight: for each query, show the highest-similarity document
print("\nBest match for each query:")
for i, query in enumerate(test_queries):
    best_idx = np.argmax(similarity_matrix[i])
    best_score = similarity_matrix[i, best_idx]
    print(f"  \"{query[:40]}...\" → doc[{best_idx}] (sim={best_score:.4f})")

In [ ]:
#@title 🎧 Listen: Reranking Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_reranking_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Before You Start: Todo Heuristic Reranker
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_todo_heuristic_reranker.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.6 Reranking — The Second Pass

Retrieval with embeddings is a fast but rough first pass. A **reranker** does a second, more careful relevance check. In production, cross-encoder models score each (query, document) pair jointly — which is slower but much more accurate than comparing independent embeddings.

We will build two rerankers:
1. A simple heuristic reranker (the TODO)
2. A cross-encoder reranker using `cross-encoder/ms-marco-MiniLM-L-6-v2`

### 🔧 TODO 2: Implement a Heuristic Reranker

In [ ]:
def heuristic_rerank(
    query: str,
    candidates: List[Tuple[str, float, int]],
) -> List[Tuple[str, float, int]]:
    """
    Rerank retrieved candidates using a simple heuristic:
    Boost the similarity score if query terms appear literally in the chunk text.

    Strategy:
        1. Tokenize the query into individual words (lowercase, ignore stopwords)
        2. For each candidate, count how many query terms appear in the document text
        3. Compute a boost factor: boost = 1 + 0.1 * (matching_terms / total_query_terms)
        4. Multiply the original similarity score by the boost factor
        5. Return candidates sorted by boosted score (highest first)

    Args:
        query: The user's query string
        candidates: List of (doc_text, similarity_score, doc_index) tuples

    Returns:
        Reranked list of (doc_text, boosted_score, doc_index) tuples
    """
    stopwords = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'do', 'does',
                 'what', 'how', 'why', 'when', 'where', 'which', 'who',
                 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of',
                 'with', 'from', 'by', 'it', 'its', 'this', 'that', 'can', 'i'}

    # ============ TODO ============
    # Step 1: Tokenize the query — split into lowercase words, remove stopwords
    # Step 2: For each candidate (doc_text, score, idx):
    #         a. Count how many query terms appear in doc_text.lower()
    #         b. Compute boost = 1 + 0.1 * (matching_terms / total_query_terms)
    #         c. Compute boosted_score = original_score * boost
    # Step 3: Sort candidates by boosted_score (descending)
    # Step 4: Return reranked list of (doc_text, boosted_score, idx)
    # ==============================

    reranked = ???  # YOUR CODE HERE

    return reranked

In [ ]:
#@title 🎧 Listen: After Todo Heuristic Reranker
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_after_todo_heuristic_reranker.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ✅ Verification: Run this cell to check your heuristic reranker

# Test case: query mentions "batch normalization"
test_candidates = [
    ("Dropout randomly deactivates neurons.", 0.85, 1),
    ("Batch normalization normalizes layer inputs.", 0.80, 0),
    ("Transfer learning reuses pre-trained models.", 0.75, 9),
]

reranked = heuristic_rerank("How does batch normalization work?", test_candidates)

# After reranking, "Batch normalization" doc should be boosted above "Dropout"
assert reranked[0][2] == 0, (
    f"Expected doc index 0 (batch norm) at top, but got index {reranked[0][2]}. "
    f"The batch norm doc should be boosted because 'batch' and 'normalization' appear in it."
)
assert reranked[0][1] > 0.80, (
    f"Expected boosted score > 0.80, got {reranked[0][1]:.4f}"
)

print("✅ Heuristic reranker works correctly!")
print(f"\nBefore reranking:")
for doc, score, idx in test_candidates:
    print(f"  [{score:.4f}] (idx={idx}) {doc}")
print(f"\nAfter reranking:")
for doc, score, idx in reranked:
    print(f"  [{score:.4f}] (idx={idx}) {doc}")

In [ ]:
#@title 🎧 Code Walkthrough: Cross Encoder Reranking
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_cross_encoder_reranking.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Cross-Encoder Reranking (Production Grade)

Now let us load a real cross-encoder model. Unlike our heuristic, a cross-encoder takes the *pair* (query, document) as input and produces a relevance score. This is much more accurate because the model can attend to interactions between query and document tokens.

In [ ]:
from sentence_transformers import CrossEncoder

print("Loading cross-encoder reranker...")
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("Cross-encoder loaded!")


def cross_encoder_rerank(
    query: str,
    candidates: List[Tuple[str, float, int]],
    model: CrossEncoder = cross_encoder
) -> List[Tuple[str, float, int]]:
    """
    Rerank candidates using a cross-encoder model.
    The cross-encoder scores each (query, document) pair jointly,
    which is more accurate than comparing independent embeddings.
    """
    if not candidates:
        return []

    # Create (query, document) pairs for the cross-encoder
    pairs = [(query, doc_text) for doc_text, _, _ in candidates]

    # Score all pairs
    cross_encoder_scores = model.predict(pairs)

    # Replace similarity scores with cross-encoder scores
    reranked = [
        (doc_text, float(ce_score), idx)
        for (doc_text, _, idx), ce_score in zip(candidates, cross_encoder_scores)
    ]

    # Sort by cross-encoder score (descending)
    reranked.sort(key=lambda x: x[1], reverse=True)
    return reranked


print("\nReady to rerank with cross-encoder!")

In [ ]:
#@title 🎧 What to Look For: Reranking Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_reranking_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: How Reranking Changes the Order

Let us see the effect of reranking on a concrete example.

In [ ]:
# Query that benefits from reranking
query = "What technique helps train very deep neural networks with 100+ layers?"
query_emb = embed_model.encode(query)

# Get top-5 from vector store (initial retrieval)
initial_results = vector_store.search(query_emb, top_k=5)

# Rerank with both methods
heuristic_results = heuristic_rerank(query, initial_results)
crossencoder_results = cross_encoder_rerank(query, initial_results)

# Visualize side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
stages = [
    ("Initial Retrieval\n(Embedding Similarity)", initial_results, '#2196F3'),
    ("After Heuristic Rerank\n(Term Matching Boost)", heuristic_results, '#4CAF50'),
    ("After Cross-Encoder Rerank\n(Joint Scoring)", crossencoder_results, '#FF5722'),
]

for ax, (title, results, color) in zip(axes, stages):
    docs_short = [textwrap.fill(doc[:70], width=40) for doc, _, _ in results]
    scores = [score for _, score, _ in results]
    indices = [idx for _, _, idx in results]

    y_pos = range(len(results))
    bars = ax.barh(y_pos, scores, color=color, alpha=0.75, edgecolor='white', height=0.6)
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f'[{idx}] {d[:35]}...' for d, idx in zip(docs_short, indices)],
                       fontsize=8)
    ax.set_xlabel('Score')
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.invert_yaxis()

    # Add score labels on bars
    for bar, score in zip(bars, scores):
        ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                f'{score:.3f}', va='center', fontsize=9)

plt.suptitle(f'Query: "{query}"', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Show which document is about residual connections (the correct answer)
print(f"\n💡 The correct answer is about residual connections (ResNet, 100+ layers).")
print(f"   Initial rank: {[idx for _, _, idx in initial_results].index(4) + 1 if 4 in [idx for _, _, idx in initial_results] else 'not in top-5'}")
print(f"   Cross-encoder rank: {[idx for _, _, idx in crossencoder_results].index(4) + 1 if 4 in [idx for _, _, idx in crossencoder_results] else 'not in top-5'}")

In [ ]:
#@title 🎧 Code Walkthrough: Context Assembly
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_context_assembly.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.7 Context Assembly

The final step before the LLM sees anything: assembling the retrieved chunks into a structured context. We use XML tags to clearly delineate sections — this helps the LLM understand the role of each piece of information.

In [ ]:
def assemble_context(
    query: str,
    retrieved_docs: List[Tuple[str, float, int]],
    system_prompt: str = "You are a helpful ML tutor. Answer based on the provided context.",
    max_context_chars: int = 4000,
) -> str:
    """
    Assemble the final context string with XML tags.

    Structure:
        <system> ... </system>
        <retrieved_context>
            <chunk relevance="0.85"> ... </chunk>
            ...
        </retrieved_context>
        <query> ... </query>
    """
    context_parts = []

    # System prompt
    context_parts.append(f"<system>\n{system_prompt}\n</system>")

    # Retrieved context
    context_parts.append("\n<retrieved_context>")
    chars_used = sum(len(p) for p in context_parts)

    for doc_text, score, idx in retrieved_docs:
        chunk_str = f'  <chunk relevance="{score:.4f}">\n    {doc_text}\n  </chunk>'
        if chars_used + len(chunk_str) > max_context_chars:
            context_parts.append("  <!-- Additional chunks truncated to fit context budget -->")
            break
        context_parts.append(chunk_str)
        chars_used += len(chunk_str)

    context_parts.append("</retrieved_context>")

    # User query
    context_parts.append(f"\n<query>\n{query}\n</query>")

    return "\n".join(context_parts)


# Demo: assemble context for our residual connections query
demo_query = "What technique helps train very deep neural networks with 100+ layers?"
demo_results = cross_encoder_rerank(demo_query, vector_store.search(embed_model.encode(demo_query), top_k=3))
demo_context = assemble_context(demo_query, demo_results)

print("=" * 70)
print("ASSEMBLED CONTEXT (what the LLM would see)")
print("=" * 70)
print(demo_context)
print("=" * 70)
print(f"\nTotal characters: {len(demo_context)}")
print(f"Estimated tokens: ~{len(demo_context) // 4}")

In [ ]:
#@title 🎧 Listen: Exp1 Chunk Size
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_25_exp1_chunk_size.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn — Experiments

### 🔧 Experiment 1: Chunk Size Matters

Let us prove empirically that chunk size affects retrieval quality. We will take the longer documents, chunk them at different sizes, embed each set of chunks, and measure how well we retrieve the right information.

In [ ]:
# Combine long docs into a single "corpus" for chunking experiments
corpus = "\n\n".join(long_docs)

# Chunk at three different sizes
chunk_configs = {
    'Small (100 chars)': chunk_fixed_size(corpus, chunk_size=100, overlap=20),
    'Medium (300 chars)': chunk_fixed_size(corpus, chunk_size=300, overlap=60),
    'Large (600 chars)': chunk_fixed_size(corpus, chunk_size=600, overlap=100),
}

# Test queries with known ground-truth topics
experiment_queries = [
    ("What is batch normalization?", "batch normalization"),
    ("How does dropout regularize?", "dropout"),
    ("Explain self-attention in transformers", "transformer"),
    ("What are residual connections?", "residual"),
    ("How does transfer learning work?", "transfer learning"),
]

print("Chunk size experiment: embedding all chunk configurations...")
chunk_results = {}

for config_name, chunks in chunk_configs.items():
    # Embed chunks
    chunk_embs = embed_model.encode(chunks)

    # For each query, find the top-1 result and check if it contains the keyword
    hits = 0
    for query, keyword in experiment_queries:
        q_emb = embed_model.encode(query)
        sims = cosine_similarity(q_emb, chunk_embs)
        top_idx = np.argmax(sims)
        top_chunk = chunks[top_idx]

        if keyword.lower() in top_chunk.lower():
            hits += 1

    accuracy = hits / len(experiment_queries) * 100
    chunk_results[config_name] = {
        'num_chunks': len(chunks),
        'avg_size': np.mean([len(c) for c in chunks]),
        'accuracy': accuracy,
        'chunks': chunks,
    }
    print(f"  {config_name}: {len(chunks)} chunks, avg {np.mean([len(c) for c in chunks]):.0f} chars, accuracy {accuracy:.0f}%")

print("\nDone!")

In [ ]:
#@title 🎧 What to Look For: Chunk Size Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_26_chunk_size_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Chunk Size vs Retrieval Quality

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

config_names = list(chunk_results.keys())
colors = ['#e74c3c', '#3498db', '#2ecc71']

# Panel 1: Number of chunks
ax1 = axes[0]
num_chunks = [chunk_results[n]['num_chunks'] for n in config_names]
bars1 = ax1.bar(range(len(config_names)), num_chunks, color=colors, alpha=0.8, edgecolor='white')
ax1.set_xticks(range(len(config_names)))
ax1.set_xticklabels([n.split('(')[0].strip() for n in config_names], fontsize=10)
ax1.set_ylabel('Number of Chunks')
ax1.set_title('How Many Chunks?', fontweight='bold')
for bar, val in zip(bars1, num_chunks):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             str(val), ha='center', fontweight='bold')

# Panel 2: Average chunk size
ax2 = axes[1]
avg_sizes = [chunk_results[n]['avg_size'] for n in config_names]
bars2 = ax2.bar(range(len(config_names)), avg_sizes, color=colors, alpha=0.8, edgecolor='white')
ax2.set_xticks(range(len(config_names)))
ax2.set_xticklabels([n.split('(')[0].strip() for n in config_names], fontsize=10)
ax2.set_ylabel('Avg Characters per Chunk')
ax2.set_title('How Big Are Chunks?', fontweight='bold')
for bar, val in zip(bars2, avg_sizes):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val:.0f}', ha='center', fontweight='bold')

# Panel 3: Retrieval accuracy
ax3 = axes[2]
accuracies = [chunk_results[n]['accuracy'] for n in config_names]
bars3 = ax3.bar(range(len(config_names)), accuracies, color=colors, alpha=0.8, edgecolor='white')
ax3.set_xticks(range(len(config_names)))
ax3.set_xticklabels([n.split('(')[0].strip() for n in config_names], fontsize=10)
ax3.set_ylabel('Top-1 Retrieval Accuracy (%)')
ax3.set_title('How Accurate Is Retrieval?', fontweight='bold')
ax3.set_ylim(0, 110)
for bar, val in zip(bars3, accuracies):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val:.0f}%', ha='center', fontweight='bold')

plt.suptitle('Chunk Size Experiment: Smaller Isn\'t Always Better',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Key takeaway: Very small chunks lose context, very large chunks")
print("   dilute the signal. The medium-sized chunks tend to perform best.")
print("   This matches the article's recommendation: 400-1000 tokens per chunk.")

In [ ]:
#@title 🎧 Listen: Exp2 Top K
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_27_exp2_top_k.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Experiment 2: Top-K Selection

How many chunks should we retrieve? Let us measure how retrieval quality changes with K.

In [ ]:
# Use our full knowledge base for this experiment
queries_for_topk = [
    ("How does batch normalization work?", {0, 10}),       # indices of relevant docs
    ("What prevents overfitting?", {1, 11}),                # dropout docs
    ("Explain the transformer architecture", {2, 12}),      # transformer docs
    ("What are residual connections?", {4, 13}),             # residual docs
    ("How does transfer learning work?", {9, 14}),           # transfer learning docs
]

k_values = [1, 2, 3, 5, 7, 10]
recall_at_k = []
precision_at_k = []

for k in k_values:
    total_recall = 0
    total_precision = 0

    for query, relevant_indices in queries_for_topk:
        q_emb = embed_model.encode(query)
        results = vector_store.search(q_emb, top_k=k)
        retrieved_indices = {idx for _, _, idx in results}

        # Recall: what fraction of relevant docs did we retrieve?
        hits = len(retrieved_indices & relevant_indices)
        recall = hits / len(relevant_indices)
        total_recall += recall

        # Precision: what fraction of retrieved docs are relevant?
        precision = hits / k
        total_precision += precision

    recall_at_k.append(total_recall / len(queries_for_topk))
    precision_at_k.append(total_precision / len(queries_for_topk))

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(k_values, recall_at_k, 'o-', color='#2196F3', linewidth=2.5,
        markersize=8, label='Recall @ K', zorder=3)
ax.plot(k_values, precision_at_k, 's-', color='#FF5722', linewidth=2.5,
        markersize=8, label='Precision @ K', zorder=3)

ax.set_xlabel('Top-K (number of retrieved chunks)', fontsize=13)
ax.set_ylabel('Score', fontsize=13)
ax.set_title('The Precision–Recall Tradeoff in Top-K Retrieval', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.set_ylim(-0.05, 1.05)
ax.set_xticks(k_values)
ax.grid(True, alpha=0.3)

# Annotate the sweet spot
sweet_k_idx = np.argmax(np.array(recall_at_k) + np.array(precision_at_k))
ax.axvline(x=k_values[sweet_k_idx], color='gray', linestyle='--', alpha=0.5)
ax.annotate(f'Sweet spot: K={k_values[sweet_k_idx]}',
            xy=(k_values[sweet_k_idx], (recall_at_k[sweet_k_idx] + precision_at_k[sweet_k_idx])/2),
            xytext=(k_values[sweet_k_idx]+1.5, 0.5),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=11, fontweight='bold', color='gray')

plt.tight_layout()
plt.show()

print(f"\n💡 As K increases, recall goes up (we find more relevant docs)")
print(f"   but precision goes down (more noise). The sweet spot is K={k_values[sweet_k_idx]}.")

In [ ]:
#@title 🎧 Code Walkthrough: Full Rag Pipeline
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_28_full_rag_pipeline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

Now let us build the complete end-to-end RAG pipeline as a single class. This brings together every component we built.

In [ ]:
class RAGPipeline:
    """
    Complete RAG Pipeline: From query to assembled context.

    Pipeline stages:
        1. Embed the query
        2. Retrieve top-K candidates from vector store
        3. Rerank candidates (cross-encoder)
        4. Assemble structured context with XML tags
    """

    def __init__(
        self,
        embed_model: SentenceTransformer,
        cross_encoder_model: CrossEncoder,
        vector_store: SimpleVectorStore,
        top_k_retrieve: int = 5,
        top_k_rerank: int = 3,
        system_prompt: str = "You are a helpful ML tutor. Answer based on the provided context.",
    ):
        self.embed_model = embed_model
        self.cross_encoder = cross_encoder_model
        self.vector_store = vector_store
        self.top_k_retrieve = top_k_retrieve
        self.top_k_rerank = top_k_rerank
        self.system_prompt = system_prompt

    def run(self, query: str, verbose: bool = True) -> Dict:
        """
        Execute the full RAG pipeline.

        Returns a dict with:
            - query: the original query
            - retrieved: initial retrieval results
            - reranked: cross-encoder reranked results
            - context: the final assembled context string
            - timings: time spent on each stage
        """
        timings = {}

        # Stage 1: Embed the query
        t0 = time.time()
        query_embedding = self.embed_model.encode(query)
        timings['embed'] = time.time() - t0

        # Stage 2: Retrieve top-K candidates
        t0 = time.time()
        retrieved = self.vector_store.search(query_embedding, top_k=self.top_k_retrieve)
        timings['retrieve'] = time.time() - t0

        # Stage 3: Rerank with cross-encoder
        t0 = time.time()
        reranked = cross_encoder_rerank(query, retrieved, self.cross_encoder)
        # Keep only top_k_rerank after reranking
        reranked = reranked[:self.top_k_rerank]
        timings['rerank'] = time.time() - t0

        # Stage 4: Assemble context
        t0 = time.time()
        context = assemble_context(query, reranked, self.system_prompt)
        timings['assemble'] = time.time() - t0

        result = {
            'query': query,
            'retrieved': retrieved,
            'reranked': reranked,
            'context': context,
            'timings': timings,
        }

        if verbose:
            total_time = sum(timings.values())
            print(f"\n{'='*70}")
            print(f"RAG Pipeline Results for: \"{query}\"")
            print(f"{'='*70}")
            print(f"\nStage timings:")
            for stage, t in timings.items():
                print(f"  {stage:>10s}: {t*1000:6.1f} ms")
            print(f"  {'total':>10s}: {total_time*1000:6.1f} ms")
            print(f"\nTop {self.top_k_rerank} reranked results:")
            for rank, (doc, score, idx) in enumerate(reranked, 1):
                doc_short = doc[:80] + '...' if len(doc) > 80 else doc
                print(f"  {rank}. [{score:.4f}] {doc_short}")
            print(f"\nAssembled context: {len(context)} chars (~{len(context)//4} tokens)")

        return result


# Create the pipeline
rag_pipeline = RAGPipeline(
    embed_model=embed_model,
    cross_encoder_model=cross_encoder,
    vector_store=vector_store,
    top_k_retrieve=5,
    top_k_rerank=3,
)

print("RAG Pipeline ready!")

In [ ]:
#@title 🎧 Code Walkthrough: Run Pipeline
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_29_run_pipeline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Run the Pipeline End-to-End

In [ ]:
# Run on several queries
demo_queries = [
    "How can I stabilize training in deep neural networks?",
    "What technique uses a generator and discriminator?",
    "How do modern language models handle long sequences?",
]

all_results = []
for query in demo_queries:
    result = rag_pipeline.run(query)
    all_results.append(result)
    print()

In [ ]:
#@title 🎧 What to Look For: Final Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_30_final_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Results — Final Multi-Panel Visualization

Now for the grand finale: a multi-panel visualization that shows the complete pipeline in action.

In [ ]:
# Pick one query for the detailed visualization
final_query = "How can I stabilize training in deep neural networks?"
final_result = rag_pipeline.run(final_query, verbose=False)

fig = plt.figure(figsize=(20, 14))

# --- Panel 1: Top-K Retrieved Chunks with Relevance Scores ---
ax1 = fig.add_subplot(2, 2, 1)
retrieved_docs = final_result['reranked']
doc_labels_short = [textwrap.fill(doc[:60], width=35) for doc, _, _ in retrieved_docs]
scores = [score for _, score, _ in retrieved_docs]

# Color by score intensity
norm_scores = np.array(scores)
norm_scores = (norm_scores - norm_scores.min()) / (norm_scores.max() - norm_scores.min() + 1e-8)
bar_colors = plt.cm.YlOrRd(0.3 + 0.7 * norm_scores)

bars = ax1.barh(range(len(retrieved_docs)), scores, color=bar_colors,
                edgecolor='white', height=0.6)
ax1.set_yticks(range(len(doc_labels_short)))
ax1.set_yticklabels(doc_labels_short, fontsize=8)
ax1.set_xlabel('Relevance Score (Cross-Encoder)', fontsize=10)
ax1.set_title('Panel 1: Top-K Retrieved Chunks\n(Reranked by Relevance)', fontsize=12, fontweight='bold')
ax1.invert_yaxis()
for bar, score in zip(bars, scores):
    ax1.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2,
             f'{score:.3f}', va='center', fontsize=10, fontweight='bold')

# --- Panel 2: Similarity Heatmap (Query vs All Docs) ---
ax2 = fig.add_subplot(2, 2, 2)
q_emb = embed_model.encode(final_query)
all_sims = cosine_similarity(q_emb, doc_embeddings)

# Sort for visual clarity
sorted_idx = np.argsort(all_sims)[::-1]
sorted_sims = all_sims[sorted_idx]
sorted_labels = [knowledge_base[i][:40] + '...' for i in sorted_idx]

im = ax2.imshow(sorted_sims.reshape(1, -1), cmap='YlOrRd', aspect='auto')
ax2.set_xticks(range(len(sorted_labels)))
ax2.set_xticklabels(sorted_labels, rotation=60, ha='right', fontsize=7)
ax2.set_yticks([])
ax2.set_title('Panel 2: Similarity Heatmap\n(Query vs All Documents)', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax2, label='Cosine Similarity', orientation='horizontal',
             pad=0.35, shrink=0.8)

# Annotate top-3
for i in range(min(3, len(sorted_sims))):
    ax2.text(i, 0, f'{sorted_sims[i]:.2f}', ha='center', va='center',
             fontsize=8, fontweight='bold', color='white' if sorted_sims[i] > 0.5 else 'black')

# --- Panel 3: Chunk Size Experiment ---
ax3 = fig.add_subplot(2, 2, 3)
config_names_short = ['Small\n(100)', 'Medium\n(300)', 'Large\n(600)']
experiment_colors = ['#e74c3c', '#3498db', '#2ecc71']
exp_accuracies = [chunk_results[n]['accuracy'] for n in chunk_results.keys()]
exp_num_chunks = [chunk_results[n]['num_chunks'] for n in chunk_results.keys()]

x = np.arange(len(config_names_short))
width = 0.35

bars_acc = ax3.bar(x - width/2, exp_accuracies, width, label='Accuracy (%)',
                    color=experiment_colors, alpha=0.8, edgecolor='white')

ax3_twin = ax3.twinx()
bars_num = ax3_twin.bar(x + width/2, exp_num_chunks, width, label='Num Chunks',
                         color=experiment_colors, alpha=0.3, edgecolor='gray', hatch='//')

ax3.set_xticks(x)
ax3.set_xticklabels(config_names_short)
ax3.set_ylabel('Retrieval Accuracy (%)', fontsize=10)
ax3_twin.set_ylabel('Number of Chunks', fontsize=10)
ax3.set_title('Panel 3: Chunk Size Experiment', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 120)

# Combined legend
lines1, labels1 = ax3.get_legend_handles_labels()
lines2, labels2 = ax3_twin.get_legend_handles_labels()
ax3.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=9)

for bar, val in zip(bars_acc, exp_accuracies):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val:.0f}%', ha='center', fontweight='bold', fontsize=9)

# --- Panel 4: Assembled Context (Color-Coded) ---
ax4 = fig.add_subplot(2, 2, 4)
ax4.set_xlim(0, 10)
ax4.set_ylim(0, 10)
ax4.axis('off')
ax4.set_title('Panel 4: Assembled Context Structure\n(Color-Coded by Section)', fontsize=12, fontweight='bold')

# Draw colored blocks representing XML sections
sections = [
    ('System Prompt', '#3F51B5', 9.5, 1.2,
     'You are a helpful ML tutor.\nAnswer based on the provided context.'),
    ('Retrieved Context', '#4CAF50', 7.5, 4.0, None),
    ('User Query', '#9C27B0', 2.8, 1.2, final_query),
]

for name, color, y, height, content in sections:
    rect = plt.Rectangle((0.5, y - height), 9, height,
                          linewidth=2, edgecolor=color, facecolor=color,
                          alpha=0.15)
    ax4.add_patch(rect)
    ax4.text(0.7, y - 0.15, f'<{name.lower().replace(" ", "_")}>',
             fontsize=9, fontweight='bold', color=color, family='monospace')

# Add chunk details in the retrieved context section
y_chunk = 7.0
for rank, (doc, score, idx) in enumerate(final_result['reranked'], 1):
    doc_short = doc[:65] + '...' if len(doc) > 65 else doc
    ax4.text(1.0, y_chunk, f'  Chunk {rank} [relevance={score:.3f}]',
             fontsize=8, fontweight='bold', color='#2E7D32')
    ax4.text(1.0, y_chunk - 0.4, f'  "{doc_short}"',
             fontsize=7, color='#333', style='italic')
    y_chunk -= 1.1

# Add the query text
ax4.text(1.0, 2.3, f'  "{final_query}"',
         fontsize=8, color='#6A1B9A', style='italic')

# Add token count
total_chars = len(final_result['context'])
ax4.text(5.0, 0.5, f'Total: {total_chars} chars (~{total_chars//4} tokens)',
         fontsize=10, fontweight='bold', ha='center', color='#555',
         bbox=dict(boxstyle='round,pad=0.4', facecolor='#f0f0f0', edgecolor='gray'))

plt.suptitle(f'RAG Pipeline: Complete Results\nQuery: "{final_query}"',
             fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Final Context Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_31_final_context_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### The Final Assembled Context

Let us print the actual context string that would be sent to the LLM. This is the final output of the entire pipeline.

In [ ]:
print("=" * 70)
print("FINAL ASSEMBLED CONTEXT")
print("This is exactly what the LLM would see in its context window.")
print("=" * 70)
print()

context = final_result['context']

# Color-code the output by section (using ANSI codes for terminal)
lines = context.split('\n')
for line in lines:
    if line.strip().startswith('<system') or line.strip().startswith('</system'):
        print(f"\033[94m{line}\033[0m")  # Blue
    elif line.strip().startswith('<retrieved') or line.strip().startswith('</retrieved'):
        print(f"\033[92m{line}\033[0m")  # Green
    elif line.strip().startswith('<chunk') or line.strip().startswith('</chunk'):
        print(f"\033[92m{line}\033[0m")  # Green
    elif line.strip().startswith('<query') or line.strip().startswith('</query'):
        print(f"\033[95m{line}\033[0m")  # Purple
    else:
        print(line)

print()
print("=" * 70)
print(f"Context size: {len(context)} characters, ~{len(context)//4} tokens")
print(f"Pipeline latency: {sum(final_result['timings'].values())*1000:.1f} ms total")
print("=" * 70)

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_32_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Reflection

### 🤔 What Did We Build?

We built a complete RAG pipeline from scratch:

1. **Text Chunking** — Three strategies (fixed-size, sentence-based, paragraph-based) with different tradeoffs
2. **Embedding** — Converting text to 384-dimensional vectors using sentence-transformers
3. **Cosine Similarity** — Implemented from scratch with numpy, the mathematical heart of retrieval
4. **Vector Store** — A simple but complete store with top-K search
5. **Reranking** — Both a heuristic reranker (your TODO) and a cross-encoder reranker
6. **Context Assembly** — Structured XML-tagged context ready for any LLM
7. **End-to-End Pipeline** — All components wired together in a single class

### 💡 Key Takeaways

- **Chunk size matters**: Too small = confetti with no context. Too large = wastes space. Sweet spot: 400-1000 tokens.
- **Top-K is a tradeoff**: More K = higher recall but lower precision. Find the balance for your use case.
- **Reranking is critical**: Initial embedding similarity is a rough first pass. A cross-encoder reranker significantly improves result ordering.
- **Structure helps the LLM**: XML tags (`<system>`, `<retrieved_context>`, `<query>`) make it easy for the model to understand what role each piece of information plays.

### 🎯 What Comes Next?

In the next notebook, we will explore **context failure modes** — what happens when the context is poisoned, too long, or contradictory. We will also implement the four core strategies (Write, Select, Compress, Isolate) for managing context at scale.

In [ ]:
#@title 🎧 Listen: Extension Challenges
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_33_extension_challenges.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Extension Challenges (Optional)

Try these if you want to go deeper:

In [ ]:
# Challenge 1: Implement overlap-aware sentence chunking
# Modify chunk_by_sentences so that adjacent chunks share 1 overlapping sentence.
# This preserves context at chunk boundaries.

# Challenge 2: Add metadata filtering to the vector store
# Extend SimpleVectorStore.search() to accept a filter_fn parameter
# that excludes documents based on metadata (e.g., only search "long" documents).

# Challenge 3: Implement hybrid search
# Combine embedding similarity (semantic) with BM25/TF-IDF (keyword) scores.
# Weight: final_score = alpha * embedding_sim + (1-alpha) * keyword_score
# Try different values of alpha and compare retrieval quality.

# Challenge 4: Build a context budget manager
# Create a function that takes a token budget and a list of retrieved chunks,
# and selects the maximum number of chunks that fit within the budget,
# prioritizing higher-relevance chunks.

print("Try these challenges! Modify the code above and re-run.")
print("Each challenge builds on what you already implemented.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_34_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### ✅ Notebook Complete!

You have built a complete RAG pipeline from documents to assembled context. Every component — chunking, embedding, similarity search, reranking, and context assembly — was implemented from scratch so you understand exactly what happens at each stage.

The next time you use a RAG system (whether it is LangChain, LlamaIndex, or a custom pipeline), you will know precisely what is happening under the hood. That understanding is what separates an engineer who *uses* tools from one who *builds* them.

**Estimated time spent: ~40 minutes**

---

*Part 3 of the Vizuara series on Context Engineering for LLMs*
*Next up: Part 4 — Context Failure Modes and How to Fix Them*